In [1]:
import pandas as pd

from tqdm import tqdm
import json

In [2]:
df_input = pd.read_excel('./file/1.part.xlsx',
                         header=0,
                         dtype=str).fillna('')

df_input['No'] = df_input['No'].astype(int)
df_input['Row'] = df_input['Row'].astype(int)

df_input

,No,Part Number,Type,Title,Subtitle,SKU,Price,Description 1,Description 2,Fit,Vehicles,Picture,Url,Json_Src,Json_Specification,Row,Year Make Model,Body & Trim,Emission & Transmission
0,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",1,2012 Honda Accord,"2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6","KA 5AT, KA 6MT"
1,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",2,2011 Honda Accord,"2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6","KA 5AT, KA 6MT"
2,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",3,2010 Honda Accord,"2 Door EX-L V6, 4 Door EX-L V6, 4 Door EX-V6","KA 5AT, KA 6MT, KL 6MT"
3,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",4,2009 Honda Accord,"2 Door EX, 2 Door EX-L, 2 Door EX-L V6, 2 Door...","KA 5AT, KA 5MT, KA 6MT, KL 5AT, KL 5MT, KL 6MT"
4,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",5,2008 Honda Accord,"2 Door EX, 2 Door EX-L, 2 Door EX-L V6, 2 Door...","KA 5AT, KA 5MT, KA 6MT, KL 5AT, KL 5MT, KL 6MT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6575,117,91502-SZ5-003,Clip (7MM),Honda 91502-SZ5-003 Clip (7MM),1998-2006 Honda 91502SZ5003,91502SZ5003,4.47,Splash Shield Clip,Honda,1998-2006 Honda,1998-2000 Honda Civic\n2000-2006 Honda Insight,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",6,2004 Honda Insight,3 Door DX,"KA 5MT, KA CVT"
6576,117,91502-SZ5-003,Clip (7MM),Honda 91502-SZ5-003 Clip (7MM),1998-2006 Honda 91502SZ5003,91502SZ5003,4.47,Splash Shield Clip,Honda,1998-2006 Honda,1998-2000 Honda Civic\n2000-2006 Honda Insight,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",7,2003 Honda Insight,3 Door DX,"KA 5MT, KA CVT"
6577,117,91502-SZ5-003,Clip (7MM),Honda 91502-SZ5-003 Clip (7MM),1998-2006 Honda 91502SZ5003,91502SZ5003,4.47,Splash Shield Clip,Honda,1998-2006 Honda,1998-2000 Honda Civic\n2000-2006 Honda Insight,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf...",8,2002 Honda Insight,3 Door DX,"KA 5MT, KA CVT"
6578,117,91

# 1. Vehicle

In [3]:
list_dict, grouped = [], df_input.groupby('Url')
for url, group in tqdm(grouped, desc='Progress', ncols=77):
    df_temp = group.reset_index(drop=True)

    dict_vehicle = {}
    for i in range(len(df_temp)):
        make, model, year = 'Honda', df_temp.loc[i, 'Year Make Model'].split('Honda')[1].strip(), int(df_temp.loc[i, 'Year Make Model'].split('Honda')[0].strip())

        if make not in dict_vehicle:
            dict_vehicle[make] = {model: [year]}
        elif model not in dict_vehicle[make]:
            dict_vehicle[make][model] = [year]
        elif year not in dict_vehicle[make][model]:
            dict_vehicle[make][model].append(year)

    # = = = = = = = = = = = = = = =

    list_vehicle = []
    for make, dict_model in dict_vehicle.items():
        for model, list_year in dict_model.items():
            year_begin, year_end = min(list_year), max(list_year)
            list_vehicle.append(f'{make} {model} {year_begin}' if year_begin == year_end else f'{make} {model} {year_begin}-{year_end}')

    # = = = = = = = = = = = = = = =

    list_dict.append({'Url': url,
                      'Vehicle': '\n'.join(sorted(list_vehicle))})

# = = = = = = = = = = = = = = =

df_output = pd.DataFrame(list_dict).sort_values(by=['Url'],
                                                ascending=[True],
                                                ignore_index=True)
df_output.to_excel('./file/2.vehicle.xlsx', index=False)


print()
print('Done ~')

Progress: 100%|███████████████████████████| 106/106 [00:00<00:00, 607.33it/s]


Done ~


# 2. Part

In [4]:
df_input = df_input[df_input.columns.tolist()[:15]].drop_duplicates().sort_values(by=['No'],
                                                                                  ascending=[True],
                                                                                  ignore_index=True)

df_input

,No,Part Number,Type,Title,Subtitle,SKU,Price,Description 1,Description 2,Fit,Vehicles,Picture,Url,Json_Src,Json_Specification
0,1,91501-STK-003,"Clip, Engine Hood Insulator","Honda 91501-STK-003 Clip, Engine Hood Insulator",2007-2016 Honda 91501STK003,91501STK003,0.40,Insulator Clip,Honda,2007-2016 Honda,2007-2009 Honda CR-V\n2007-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
1,2,91501-S1K-003,"Clip, Engine Hood Insulator","Honda 91501-S1K-003 Clip, Engine Hood Insulator",1998-2009 Honda 91501S1K003,91501S1K003,3.32,Insulator Clip,Honda,1998-2009 Honda,1998-2006 Honda Odyssey\n1999-2007 Honda Accor...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
2,3,91503-SZ3-003,"Clip A, Bumper","Honda 91503-SZ3-003 Clip A, Bumper",1996-2024 Honda 91503SZ3003,91503SZ3003,3.70,Under Cover Retainer Clip,Honda,1996-2024 Honda,1996-2024 Honda Odyssey\n1997-2011 Honda Accor...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
3,4,91503-SZ5-003,"Clip, Bumper Face","Honda 91503-SZ5-003 Clip, Bumper Face",1997-2015 Honda 91503SZ5003,91503SZ5003,3.62,Side Trim Retainer Clip,Honda,1997-2015 Honda,1997-2007 Honda Accord\n1997-2010 Honda Odysse...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
4,6,91561-S84-A21ZB,"Clip, RR. Panel *NH167L* (GRAPHITE BLACK)","Honda 91561-S84-A21ZB Clip, RR. Panel *NH167L*...",2002-2022 Honda 91561S84A21ZB,91561S84A21ZB,2.45,Trunk Side Trim Retainer,Honda,2002-2022 Honda,2002-2011 Honda Civic\n2003-2011 Honda Element...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,113,90700-SJ4-000,"Clip, Engine Hood Insulator","Honda 90700-SJ4-000 Clip, Engine Hood Insulator",1989-2007 Honda 90700SJ4000,90700SJ4000,2.62,Insulator Retainer,Honda,1989-2007 Honda,1989-1991 Honda Civic\n1990-1991 Honda CRX\n19...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
105,114,75305-SHJ-003,"Clip, FR. Door Protector","Honda 75305-SHJ-003 Clip, FR. Door Protector",2005-2010 Honda Odyssey 75305SHJ003,75305SHJ003,1.26,Body Side Molding Clip,Honda,2005-2010 Honda Odyssey,2005-2010 Honda Odyssey,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
106,115,90602-SFA-004,"Clip, Cowl Top","Honda 90602-SFA-004 Clip, Cowl Top",2005-2011 Honda 90602SFA004,90602SFA004,3.16,Cowl Grille Clip,Honda,2005-2011 Honda,2005-2006 Honda CR-V\n2006-2010 Honda Odyssey\...,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."
107,116,91514-SAA-003,"Clip, Side Sill Garnish (Upper)","Honda 91514-SAA-003 Clip, Side Sill Garnish (U...",2007-2013 Honda Fit 91514SAA003,91514SAA003,3.22,Rocker Molding Retainer,Upper,2007-2013 Honda Fit,2007-2013 Honda Fit,,https://www.hondapartsnow.com/genuine/honda~cl...,"{""0"": ""https://www.hondapartsnow.com/resources...","{""0"": {""Brand"": ""Genuine Honda""}, ""1"": {""Manuf..."


In [5]:
list_series, list_column, list_column_specification = [], df_input.columns.tolist(), []
for _, series in tqdm(df_input.iterrows(), total=len(df_input), desc='Progress', ncols=77):
    dict_dict = json.loads(series['Json_Specification'])
    dict_ = {k: v for dict_ in dict_dict.values() for k, v in dict_.items()}

    for key, value in dict_.items():
        if key in list_column:
            key = f'{key} (Specification)'

        if key not in list_column_specification:
            list_column_specification.append(key)

        series[key] = value

    list_series.append(series)

# = = = = = = = = = = = = = = =

df_output = pd.DataFrame(list_series).sort_values(by=['No'],
                                                  ascending=[True],
                                                  ignore_index=True)
df_output[list_column[:13] + sorted(list_column_specification) + list_column[13:]].to_excel('./file/2.part.xlsx', index=False)

print()
print('Done ~')

Progress: 100%|███████████████████████████| 109/109 [00:00<00:00, 422.07it/s]



Done ~
